In [ ]:
from utils import *
import warnings
warnings.filterwarnings('ignore')
import pymc3 as pm
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import arviz as az
import numpy as np

In [ ]:
df = get_data(
    'AAPL', 
    '2018-06-29', '2023-06-09', interval='1d')
df_og= df.copy()


In [ ]:
df['Close'].plot(figsize=(15, 5), title='AAPL Stock Price')

In [ ]:
df = get_returns(df)

In [ ]:
df.returns.plot(figsize=(15, 5), title='AAPL % Returns', xlabel='Returns' )

In [ ]:
df['moving_average'] = df['returns'].rolling(10, closed='left').mean()

# volatility std * root(T)
df['volatility'] = df['returns'].rolling(10, closed='left').std() * np.sqrt(10) * 1000

# in the previous line of coe we multiplied by 1000 to scale the data to make it easier to work with
# this allowed us to interpret the histogram plot easier and to set our prior distributions
# we rescaled it back to the original scale later on to make it easier to interpret the results

In [ ]:
df.volatility.plot(figsize=(15, 5), title='AAPL Volatility')

In [ ]:
# drop na rows
df = df.dropna()

In [ ]:
df = df[['returns', 'volatility']]

In [ ]:
def assign_volatility_cluster(df, mus=[20,60,100], sigmas=[10,10,10]):
    """
    Assigns a volatility cluster to each value in the DataFrame.
    Give a df with a column named 'Value', this should be the volatility measure of a given window. 
    Let the index be the window or any arbitrary index.
    """
    # df drop rows with nan
    df.dropna(inplace=True)
    # Prepare the data
    
    values = df['volatility'].values

    with pm.Model() as model:
        # Specify the number of clusters
        k = 3
        # Priors for the cluster parameters
        mus = pm.Normal('mus', mu=mus, sd=10, shape=k)
        sigmas = pm.HalfNormal('sigmas', sd=10, shape=k)
        weights = pm.Dirichlet('weights', a=np.ones(k))

        # Likelihood
        likelihood = pm.NormalMixture('likelihood', w=weights, mu=mus, sd=sigmas, observed=values)

        # Sample from the posterior
        trace = pm.sample(2000, tune=1000)
        
    cluster_means = np.array(trace['mus'][-1])
    diff = cluster_means - df['volatility'].values[:, np.newaxis]
    cluster = np.argmin(np.abs(diff), axis=1)
    df['cluster'] = cluster



    return df, trace

    # Extract the cluster assignments

df_clusters, trace = assign_volatility_cluster(df)

In [ ]:
pm.traceplot(trace)

In [ ]:
# df_clusters.value_counts('cluster')
df_clusters.value_counts('cluster') 

In [ ]:
# join df_clusters and df_og
df_clusters = df_clusters.join(df_og, how='left')
df_clusters.reset_index(inplace=True)
df_clusters.head()

In [ ]:
#import mcolors
from matplotlib import colors as mcolors

# Get unique categories and assign corresponding color map
unique_categories = sorted(df_clusters['cluster'].unique())  # Sort the unique categories
color_map = mcolors.ListedColormap(['#90bcdc', '#ffbc84', '#98cc94'])  # Define specific colors for the categories

# Plot scatter points with color based on cluster
norm = mcolors.Normalize(vmin=0, vmax=len(unique_categories)-1)
plt.scatter(df_clusters.index, df_clusters['High'], c=df_clusters['cluster'].astype('category').cat.codes,
            cmap=color_map, norm=norm)

# Create a legend with matching colors
legend_labels = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=color_map(norm(i)), markersize=8)
                 for i, _ in enumerate(unique_categories)]
plt.legend(legend_labels, unique_categories)

# Set labels for x and y axes
plt.xlabel('Date')
plt.ylabel('Price')
plt.title('AAPL Stock Price with Clusters labels')

# Display the plot
plt.show()


In [ ]:
from sklearn import preprocessing

label_encoder = preprocessing.LabelEncoder() 
df['cluster'] = label_encoder.fit_transform(df['cluster'])

In [ ]:
train_cluster_names = df.cluster.unique()
train_cluster_idx = df.cluster.values

n_train_types = len(df.cluster.unique())
df.volatility = df.volatility/1000

df[['cluster', 'volatility']].head()

#plot histogram of volatility
plt.title('Histogram of Volatility of AAPL Stock')
plt.hist(df.volatility, bins=30, edgecolor='black')


In [ ]:
category1_values = df[df.cluster == 0].volatility.values
category2_values = df[df.cluster == 1].volatility.values
category3_values = df[df.cluster == 2].volatility.values

category1_width=category1_values.max() - category1_values.min()
category2_width=category2_values.max() - category2_values.min()
category3_width=category3_values.max() - category3_values.min()
bin_width = 0.005
bins_1 = category1_width / bin_width
bins_2 = category2_width / bin_width
bins_3 = category3_width / bin_width
plt.figure(figsize=(8, 6))

plt.hist(category1_values, bins=int(round(bins_1)),  label='Cluster 0', color='#90bcdc')
plt.hist(category2_values, bins=int(round(bins_2)),  label='Cluster 1', color='#ffbc84')
plt.hist(category3_values, bins=int(round(bins_3)),  label='Cluster 2', color='#98cc94')

plt.xlabel('volatility')
plt.ylabel('Frequency')
plt.title('Histograms by Cluster')
plt.legend()
plt.grid(True)


In [ ]:
with pm.Model() as hierarchical_model:
    # global model parameters
    α_μ_tmp = pm.Normal('α_μ_tmp', mu=0.5, sd=0.1)
    α_σ_tmp = pm.HalfNormal('α_σ_tmp', .05)
    β_μ = pm.Normal('β_μ', mu=0.5, sd=0.1)
    β_σ = pm.HalfNormal('β_σ', .05)

    # Cluster specific model parameters
    α_tmp = pm.Normal('α_tmp', mu=α_μ_tmp, sd=α_σ_tmp, shape=3)  
    # Intercept for each Cluster, distributed around Cluster mean 
    β = pm.Normal('β', mu=β_μ, sd=β_σ, shape=3)
    # Model error
    eps = pm.HalfCauchy('eps', .5)

    return_est = α_tmp[train_cluster_idx] + β[train_cluster_idx]

    # Data likelihood
    return_like = pm.Normal('return_like', mu=return_est, sd=eps, observed=df.volatility)
    
with hierarchical_model:
    hierarchical_trace = pm.sample(2000, tune=1000, target_accept=.9)
    
pm.traceplot(hierarchical_trace, var_names=['α_μ_tmp', 'β_μ', 'α_σ_tmp', 'β_σ', 'eps'])

In [ ]:
az.plot_forest(hierarchical_trace, var_names=['α_tmp', 'β'], combined=True)

In [ ]:
ppc = pm.sample_posterior_predictive(hierarchical_trace, samples=2000, model=hierarchical_model)
az.r2_score(df.returns.values, ppc['return_like'])